**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

/Users/vadimbenichou/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?
        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Explanation of the fonction act : At a state s, the function prescribes the action to do and the environment takes the output of the function act. For instance a pawn is in a case B at time t, it could go up, down, left, rigt (a= 0 for up for instance, 1 for down and so on...) and the next position will be dictate by act. The environment gets a new state. The aim is that the function determines the best action to undertake based on previous experiences and rewards. 

Epsilon can be considered as the exploration rate. To optimise exploitation, the rate can be small or equal to 0. 
Furthermore, Epsilon is essential here because it will determine if  one picks more a stochastic policy or deterministic. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0: 
            if self.x == self.grid_size-3:
                self.x = self.x-1               # if  edge, go UP 
            else:
                self.x = self.x + 1             # go DOWN
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1               # if  edge, go DOWN
            else:
                self.x = self.x-1               # go UP 
        elif action == 2:
            if self.y == self.grid_size - 3: 
                self.y = self.y - 1             # if  edge, go LEFT
            else:
                self.y = self.y + 1             # go RIGHT
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1             # if  edge, go RIGHT
            else:
                self.y = self.y - 1             # go LEFT
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T= 200
temperature = 0.3
epochs_train = 7 # set small when debugging
epochs_test = 4 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Board__ represents all the possible positions of the mouse in the board.   
__Position__ is used to get the position of coordanates(x,y) of the mouse in the environment (the grid).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
import random
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return random.randint(0,3)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):

        state = env.reset()#to reset the game at each epoch
        
        game_over = False #to ensure the end of the game

        lose = 0
        win = 0
        

        while not game_over:
            action = agent.act(state)# To make the agent perform an action

            # to submit the environment to the action and get the next state and the reward
            # and if the games end
            previousstate = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(previousstate, state,  action, reward, game_over)

            
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/17.0. Average score (-7.0)
Win/lose count 5.0/12.0. Average score (-7.0)
Win/lose count 7.0/8.0. Average score (-5.0)
Win/lose count 10.5/9.0. Average score (-3.375)
Final score: -3.375


***
## DQN

##A  FAIRE

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




This formula is provided  
$$
\begin{aligned}
Q^{\pi}(s,a) &= E_{\pi}[ \sum_{t=0}^{\infty}\gamma^t r(s_t,a_t) \ | \ s_0 = s, a_0 = a]\\
&= E[r(s,a)]+ \gamma E_{\pi}[ \sum_{t=1}^{\infty}\gamma^{t-1} r(s_{t},a_{t}) \ | \ s_0 = s, a_0 = a ]    \\
&= E[r(s,a)]+ \gamma \sum_{s'\in S}^{} \sum_{a' \in A}^{} p(s_1 = s',a_1 = a'|s_0 = s, a_0 = a)E_{\pi}[\sum_{t=1}^{\infty}\gamma^{t-1} r(s_{t},a_{t}) \ |\  s_1 = s',a_1 = a' ]\\
& E_{\pi}[\sum_{t=1}^{\infty}\gamma^{t-1} r(s_{t},a_{t}) \ |\  s_1 = s',a_1 = a' ] = Q^{\pi}(s',a') \; \text{so we have:}  \\
& = E[r(s,a)]+ \gamma \sum_{s'\in S}^{} \sum_{a' \in A}^{} p(s_1 = s',a_1 = a'|s_0 = s, a_0 = a)Q^{\pi}(s',a')\\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{aligned}
$$

Hence $ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $


Let's say that the optimal policy $\pi^*$ exists with $Q^{*}(s,a) = max_{\pi} Q^{\pi}(s,a)$.

we can write:

$$
\begin{aligned}
Q^*(s,a) & = \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
 & = r(s,a) + \gamma \max_{\pi} [ \sum_{s' \in S}^{}  \sum_{a' \in A}^{} p(s',a'|s,a)Q^{\pi}(s',a') ] \\
 & = r(s,a) + \gamma  \sum_{s' \in S}^{}p(s'|s,a)
 \max_{\pi} [\sum_{a' \in A}^{} \pi(a'|s')Q^{\pi}(s',a') ]\\
& = r(s,a) + \gamma  \sum_{s' \in S}^{}p(s'|s,a) \max_{a'} Q^*(s',a')\\
& = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{aligned}
$$

Hence $Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, memory, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, memo):
        self.memory.append(memo)
        
        if len(self.memory) > self.max_memory:
            del self.memory[0]
        
        pass

    def random_access(self):
        batch = random.choice(self.memory)
        return batch

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__()

        # Discount for Q learning
        self.discount = 0.75
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

               

    def learned_act(self, s):
        
        act = self.model.predict(np.array([s],ndmin=2)) 
        return np.argmax(act[0])

    def reinforce(self, s_, n_s, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4)) # 4 actions to take
        
        
        #for state, action, reward, next_state, done in minibatch:
        for i in range(self.batch_size):
            
            # creation of a batchof past experiences:
            batch = self.memory.random_access()
            
            # each row of mini_batch has: state, next_state, action, reward, game_over_
            s_ , n_s_, a_, r_, game_over_ = batch

            input_states[i] = s_
            
            if game_over_:
                target_q[i, a_] = r_ 
                
            else:
                x = np.reshape(n_s_, (1, 5, 5, self.n_state)) 
                target_q[i, a_] = r_ + self.discount * np.amax(self.model.predict(x)) 
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q) # loss function. 


        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(64, input_dim=5*5*self.n_state))
        model.add(Dense(4))
        
        model.compile(sgd(lr=0.1, decay=0.0001, momentum=0.0), "mse")
        self.model = model

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/007 | Loss 0.0067 | Win/lose count 4.5/6.0 (-1.5)
Epoch 001/007 | Loss 0.0087 | Win/lose count 12.5/1.0 (11.5)
Epoch 002/007 | Loss 0.0027 | Win/lose count 2.0/1.0 (1.0)
Epoch 003/007 | Loss 0.0026 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/007 | Loss 0.0088 | Win/lose count 5.0/1.0 (4.0)
Epoch 005/007 | Loss 0.0008 | Win/lose count 7.0/6.0 (1.0)
Epoch 006/007 | Loss 0.0021 | Win/lose count 6.0/2.0 (4.0)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
from keras.layers import Flatten

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50, (2, 2), input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(10, (2, 2)))
        
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 10, prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/010 | Loss 0.0127 | Win/lose count 9.5/12.0 (-2.5)
Epoch 001/010 | Loss 0.0241 | Win/lose count 6.5/8.0 (-1.5)
Epoch 002/010 | Loss 0.0141 | Win/lose count 7.0/7.0 (0.0)
Epoch 003/010 | Loss 0.0009 | Win/lose count 8.5/1.0 (7.5)
Epoch 004/010 | Loss 0.0012 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/010 | Loss 0.0093 | Win/lose count 9.5/6.0 (3.5)
Epoch 006/010 | Loss 0.0003 | Win/lose count 3.5/3.0 (0.5)
Epoch 007/010 | Loss 0.0024 | Win/lose count 7.0/1.0 (6.0)
Epoch 008/010 | Loss 0.0005 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/010 | Loss 0.0003 | Win/lose count 5.5/4.0 (1.5)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 12.5/11.0. Average score (1.5)
Win/lose count 9.0/2.0. Average score (4.25)
Win/lose count 9.5/2.0. Average score (5.333333333333333)
Win/lose count 3.5/0. Average score (4.875)
Final score: 4.875
Test of the FC
Win/lose count 5.0/8.0. Average score (-3.0)
Win/lose count 2.5/4.0. Average score (-2.25)
Win/lose count 5.0/4.0. Average score (-1.1666666666666667)
Win/lose count 4.5/5.0. Average score (-1.0)
Final score: -1.0


In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.05)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/1.0. Average score (1.0)
Win/lose count 1.5/0. Average score (1.25)
Win/lose count 2.0/1.0. Average score (1.1666666666666667)
Win/lose count 1.0/1.0. Average score (0.875)
Final score: 0.875
Test of the FC
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0.5/0. Average score (-0.25)
Win/lose count 1.5/2.0. Average score (-0.3333333333333333)
Win/lose count 3.0/1.0. Average score (0.25)
Final score: 0.25


In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 18.0/2.0. Average score (16.0)
Win/lose count 8.0/7.0. Average score (8.5)
Win/lose count 16.0/8.0. Average score (8.333333333333334)
Win/lose count 12.5/6.0. Average score (7.875)
Final score: 7.875
Test of the FC
Win/lose count 5.5/4.0. Average score (1.5)
Win/lose count 15.0/13.0. Average score (1.75)
Win/lose count 8.5/3.0. Average score (3.0)
Win/lose count 13.5/6.0. Average score (4.125)
Final score: 4.125


In [18]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 37.5/1.0. Average score (36.5)
Win/lose count 22.0/2.0. Average score (28.25)
Win/lose count 28.0/1.0. Average score (27.833333333333332)
Win/lose count 32.5/3.0. Average score (28.25)
Final score: 28.25
Test of the FC
Win/lose count 22.0/1.0. Average score (21.0)
Win/lose count 18.5/4.0. Average score (17.75)
Win/lose count 13.5/1.0. Average score (16.0)
Win/lose count 7.5/2.0. Average score (13.375)
Final score: 13.375


We can notice that with a low temperature level (0.05) the results are not really concluant. 
It seems that the CNN performs better than the FC, but the more we increase the temperature, the deeper the gap.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            previousstate = state
            #state, reward, game_over = env.act(action)
            state, reward, game_over = env.act(action, train=True)

            
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward # Update the counters

            # Apply the reinforcement strategy
            loss = agent.reinforce(previousstate, state, action, reward, game_over)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        # define borders
        self.position[0:2,:]= -1 # first two lines
        self.position[:,0:2] = -1 # first two columns
        self.position[-2:, :] = -1 # last two lines
        self.position[:,-2:] = -1 # last two columns

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 10, prefix='cnn_train_explore')

Epoch 001/010 | Loss 0.0067 | Win/lose count 7.5/26.100000000000072 (-18.600000000000072)
Epoch 002/010 | Loss 0.0134 | Win/lose count 9.8/23.200000000000035 (-13.400000000000034)
Epoch 003/010 | Loss 0.0084 | Win/lose count 11.800000000000004/23.300000000000043 (-11.500000000000039)
Epoch 004/010 | Loss 0.0104 | Win/lose count 9.200000000000003/20.800000000000033 (-11.60000000000003)
Epoch 005/010 | Loss 0.0034 | Win/lose count 7.000000000000001/23.40000000000007 (-16.40000000000007)
Epoch 006/010 | Loss 0.0040 | Win/lose count 7.300000000000002/20.30000000000003 (-13.000000000000028)
Epoch 007/010 | Loss 0.0045 | Win/lose count 11.400000000000002/16.499999999999968 (-5.099999999999966)
Epoch 008/010 | Loss 0.0140 | Win/lose count 9.600000000000003/21.700000000000017 (-12.100000000000014)
Epoch 009/010 | Loss 0.0026 | Win/lose count 6.600000000000002/22.500000000000064 (-15.900000000000063)
Epoch 010/010 | Loss 0.0045 | Win/lose count 7.3000000000000025/23.000000000000075 (-15.7000000

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')

Win/lose count 10.100000000000003/21.60000000000002. Average score (-11.500000000000016)
Win/lose count 14.800000000000008/17.39999999999998. Average score (-7.0499999999999945)
Win/lose count 8.000000000000002/28.10000000000009. Average score (-11.400000000000025)
Win/lose count 8.400000000000002/22.000000000000036. Average score (-11.950000000000028)
Final score: -11.950000000000028


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***